In [1]:
import os
import cv2
import pyttsx3
import speech_recognition as sr
import threading
import numpy as np
import pyaudio
from keras.models import model_from_json
import numpy as np
import New_gemini
import face_recognition
import json
# Load the JSON file
with open("lowercase_diseases_medicines.json", "r") as json_file:
    diseases_medicines = json.load(json_file)

json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")
haar_file=cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade=cv2.CascadeClassifier(haar_file)


In [ ]:
class Assiestant:
    def __init__(self):
        
        self.cap=None
        self.camera_thread = None
        self.camera_thread1 = None
        self.camera_thread3 = None
        self.Emotion="no" 
        self.path = "photos"
        self.mylist = os.listdir(self.path)
        self.doc="no doctor"
        self.gemin="no"
        
    def init_camera(self):
        self.cap=cv2.VideoCapture(0)
        self.camera_thread=threading.Thread(target=self.onlyCam)
        self.camera_thread.start()
    def enImage(self,Image):
        enList=[]
        for img in Image:
            facLoc=face_recognition.face_locations(img)
            enclid = face_recognition.face_encodings(img)[0]
            enList.append(enclid)
        return enList  
    def Name_count(self):
        print("NAME count")
        self.name = []
        self.className = []
        for cl in self.mylist:
            imlist=cv2.imread(f'{self.path}/{cl}')
            #imlist=cv2.cvtColor(imlist, cv2.COLOR_RGB2GRAY)
            self.name.append(imlist)
            self.className.append(os.path.splitext(cl)[0]) 
        #print("this is",self.name)
        self.enKnown = self.enImage(self.name)
        print(self.enKnown)
        self.cap =cv2.VideoCapture(0)
        self.camera_thread3=threading.Thread(target=self.Face_recog)
        self.camera_thread3.start()
        #print(self.enKnown)
        
        
    def timeCount(self,name):
        with open("attandence.csv","r+") as f: 
            myDataList = f.readline()
            nameList =[]
            for line in myDataList:
                enty = line.split(",")
                nameList.append(enty[0])
                if name not in nameList:
                    now = datetime.now()
                    fortime = now.strftime("%H:%M:%S")
                    f.writelines(f"\n {name},{fortime}")
                    break
    def Face_recog(self):
        print("Start face recognition")
        while True:
            t, imgFir= self.cap.read()
            imgFlp=cv2.flip(imgFir,1)
            img=cv2.cvtColor(imgFlp, cv2.COLOR_RGB2BGR)
            imgs=img
            faceCurr = face_recognition.face_locations(imgs)
            enCurr = face_recognition.face_encodings(imgs,faceCurr)
            for i,j in zip(enCurr,faceCurr):
                k=np.array(i)
                match = face_recognition.compare_faces(self.enKnown,k)
                facDis = face_recognition.face_distance(self.enKnown,k)
                positionName=np.argmin(facDis)
                if match[positionName]:
                    nameF=self.className[positionName]
                    #print(nameF)
                    #print(type(nameF))
                    x1,x2,x3,x4=j
                    cv2.rectangle(img ,(x4,x1),(x2,x3),(255,0,255),2)
                    cv2.putText(img,nameF,(x4+6,x3-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,100,0),1)
                    #timeCount(nameF)
                    
                else:
                    x1,x2,x3,x4=j
                    #print(x1,x2,x3,x4)
                    cv2.rectangle(img ,(x4,x1),(x2,x3),(255,0,255),2)
                    cv2.putText(img,"Unkonwn",(x4+6,x3-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,128,0),2)
            cv2.imshow("Output",img)
            if cv2.waitKey(1) & 0xFF==ord("q"):
                self.release_camera()
                break
                    
                    
    def release_camera(self):
        if self.cap is not None:
            self.cap.release()
            cv2.destroyAllWindows()
            self.cap=None
            self.camera_thread.join()
            self.camera_thread = None
            self.camera_thread1=None
            camera_thread3=None
            self.Emotion="no"
        
        
    def Gemini(self,com_m):
        print("run gemini") 
        df=New_gemini.Text(com_m)
        sentences = df.split('.')[:3]
        jk=[]
        for i in sentences:
            new_str = i.replace('\n', '')
            new_str = i.replace('**', '')
            new_str = i.replace('##', '')
            new_str = i.replace(':', '')
            jk.append(new_str)
        li=' '.join(jk)
        print(li)
        engine.say(li)
        # Play the speech
        engine.runAndWait()
        
    def Doctor_voice(self,text_voice):
        disease_name = text_voice  # Standardize input
        if disease_name in diseases_medicines:
            medicines = diseases_medicines[disease_name]
            print(f"Medicines for {disease_name}: {', '.join(medicines)}")
            engine.say(medicines)
            # Play the speech
            engine.runAndWait()
        else:
            print(f"Disease '{disease_name}' not found in the database.")
        
    def analyze_command2(self, command):
        print("analyzed command2 activation:",command) 
        if self.doc=="no doctor":
            if "on camera" in command.lower():
                if not self.cap or not self.cap.isOpened():
                    self.init_camera()
            elif "off camera" in command.lower():
                self.release_camera() 
            elif "face" in command.lower():
                self.Name_count()
                print("face recogbition")
            elif "emotion" in command.lower():
                self.Emotion="yes"
            elif "no emotion" in command.lower():
                self.Emotion="no"
                self.camera_thread1=None
                print("emotion off")
        elif self.doc=="doctor": 
            text=command.lower()
            self.Doctor_voice(text)
        elif self.gemin=="yes":    
            sf=command.lower()
            print("result: ",sf)
            self.Gemini(sf)
            self.gemin=="no"
        else:
            print("No Command2 mode")
            
    def analyze_command(self, command):
        if "on doctor" in command:
            self.doc="doctor"
            self.gemin="no"
            self.analyze_command2(command)
        elif "off doctor" in command.lower() or "of doctor" in command.lower():
            self.doc="no doctor"
            self.gemin="no"
            self.analyze_command2(command)
        elif "on gemini" in command.lower(): 
            self.gemin="yes"
            self.doc="other"
            print("Active Gemini mode")
        else:
            print("No Command1 mode")
            self.analyze_command2(command)
            
            
            
    def camera_open(self):
        while self.cap and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                cv2.imshow("Output",frame)
                if cv2.waitKey(1) & 0xFF==ord("q"):
                    self.release_camera()
                    break
    
    def onlyCam(self):
        while self.cap and self.cap.isOpened():
            self.ret, self.frame = self.cap.read() 
            if self.Emotion=="yes":
                self.camera_thread1=threading.Thread(target=self.camera_feed)
                self.camera_thread1.start()
                if cv2.waitKey(1) & 0xFF==ord("q"):
                    self.release_camera()
                    break
            else:
                cv2.imshow("Output",self.frame)
                if cv2.waitKey(1) & 0xFF==ord("q"):
                    self.release_camera()
                    break
    def camera_feed(self):
            labels = {0 : 'angry', 1 : 'disgust', 2 : 'fear', 3 : 'happy', 4 : 'neutral', 5 : 'sad', 6 : 'surprise'}
            gray=cv2.cvtColor(self.frame,cv2.COLOR_BGR2GRAY)
            faces=face_cascade.detectMultiScale(self.frame,1.3,5)
            try: 
                for (p,q,r,s) in faces:
                    image = gray[q:q+s,p:p+r]
                    cv2.rectangle(self.frame,(p,q),(p+r,q+s),(255,0,0),2)
                    image = cv2.resize(image,(48,48))
                    feature = np.array(image)
                    img = feature.reshape(1,48,48,1)
                    pred = model.predict(img)
                    prediction_label = labels[pred.argmax()]
                    # print("Predicted Output:", prediction_label)
                    # cv2.putText(im,prediction_label)
                    cv2.putText(self.frame, '% s' %(prediction_label), (p-10, q-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,2, (0,0,255))
                cv2.imshow("Output",self.frame)
                cv2.waitKey(1)
            except cv2.error:
                pass

            
# Initialize the TTS engine
engine = pyttsx3.init()
# Text to be converted
text ="Hello, my name is tarbo .i am a doctor robots."
# Convert text to speech
engine.say(text)
# Play the speech
engine.runAndWait()


a=Assiestant()
#a.analyze_command("on camera")

recognizer = sr.Recognizer()
with sr.Microphone() as source:
    print("Listening for commands...")
    recognizer.adjust_for_ambient_noise(source)
    while True:
        print("Listening...")
        try:
            audio = recognizer.listen(source)  # Set timeout to 1 second
            command = recognizer.recognize_google(audio)
            print("Command:", command)
            a.analyze_command(command)
        except:
            print("no command detected")


Listening for commands...
Listening...
